In [138]:
import pandas as pd
import os
import numpy as np

csv_files = [f for f in os.listdir('../data/raw') if f.endswith('.csv')]

In [139]:
df_dirty = pd.read_csv(f'../data/raw/{csv_files[1]}')

In [122]:
'''# displaying all three links (se_link, n3k_link, bs_link) if they are not identical
df_dirty.loc[(df_dirty['se_link'] != df_dirty['n3k_link']) | (df_dirty['se_link'] != df_dirty['bs_link'])]'''

In [141]:
# Removing all '\n' and '\t' from n3k_title, n3k_body, bs_title, bs_body as well as leading and trailing whitespaces and more than
# one whitespace in between words and replacing them with np.nan if they are empty strings
for col in ['n3k_title', 'n3k_body', 'bs_title', 'bs_body']:
    df_dirty[col] = df_dirty[col].str.replace('\n', ' ').str.replace('\t', ' ').str.replace(' +', ' ').str.strip()
    df_dirty.loc[df_dirty[col] == '', col] = np.nan    

# replacing n3k_titles with NA if they contain certain words and are less than 20 characters
for word in ['robot', 'subscribe', 'register']:
    df_dirty.loc[(df_dirty['n3k_title'].str.lower().str.contains(word))
                 & (df_dirty['n3k_title'].str.len() < 20), 'n3k_title'] = np.nan
    
# doing the same with n3k_body
for word in ['cookies', 'javascript', 'register', 'explorer', 'benzinga', 'djreprints']:
    df_dirty.loc[(df_dirty['n3k_body'].str.lower().str.contains(word))
                 & (df_dirty['n3k_body'].str.len() < 400), 'n3k_body'] = np.nan

# Doing the same for bs_title and bs_body
for word in ['yahoo finance', 'bloomberg', 'yahoo news', 'navigation', 'the straits times']:
    df_dirty.loc[(df_dirty['bs_title'].str.lower().str.contains(word))
                 & (df_dirty['bs_title'].str.len() < 20), 'bs_title'] = np.nan

for word in ['javascript', 'copyright', 'benzinga', 'djreprints']:
    df_dirty.loc[(df_dirty['bs_body'].str.lower().str.contains(word))
                 & (df_dirty['bs_body'].str.len() < 400), 'bs_body'] = np.nan

/var/folders/h4/tbxkzqv53pd3f16_r3jbmshw0000gn/T/ipykernel_5381/3117847292.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_dirty[col] = df_dirty[col].str.replace('\n', ' ').str.replace('\t', ' ').str.replace(' +', ' ').str.strip()


In [142]:
# Creating a column with the length of the n3k_body and bs_body
df_dirty['n3k_body_len'] = df_dirty['n3k_body'].astype(str).str.len()
df_dirty['bs_body_len'] = df_dirty['bs_body'].astype(str).str.len()

# Creating a column with the length of the n3k_title, bs_title and se_title
df_dirty['n3k_title_len'] = df_dirty['n3k_title'].astype(str).str.len()
df_dirty['bs_title_len'] = df_dirty['bs_title'].astype(str).str.len()
df_dirty['se_title_len'] = df_dirty['se_title'].astype(str).str.len()

# Creating and empty column for title and body
df_dirty['title'] = ''
df_dirty['body'] = ''

# Filling the title and body columns with the longer title and body
df_dirty.loc[df_dirty['n3k_body_len'] > df_dirty['bs_body_len'], 'body'] = df_dirty['n3k_body']
df_dirty.loc[df_dirty['n3k_body_len'] < df_dirty['bs_body_len'], 'body'] = df_dirty['bs_body']
df_dirty.loc[df_dirty['n3k_title_len'] > df_dirty['bs_title_len'], 'title'] = df_dirty['n3k_title']
df_dirty.loc[df_dirty['n3k_title_len'] < df_dirty['bs_title_len'], 'title'] = df_dirty['bs_title']

# creating a column with the length of the title
df_dirty['title_len'] = df_dirty['title'].astype(str).str.len()

# Filling the title and body columns with the se_title if they are longer than the title
df_dirty.loc[df_dirty['se_title_len'] > df_dirty['title_len'], 'title'] = df_dirty['se_title']

# Dropping the columns that are not needed anymore
df_dirty.drop(['n3k_title', 'n3k_body', 'bs_title', 'bs_body', 'n3k_body_len', 'bs_body_len',
            'n3k_title_len', 'bs_title_len', 'se_title_len', 'se_title', 'title_len'], axis=1, inplace=True)


In [143]:
# Dropping all rows that have no title or body
df_dirty.dropna(subset=['title', 'body'], inplace=True)

In [144]:
showme = df_dirty[['title', 'body']]